In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
import base64

# ============================================================================
# GLOBAL STATE MANAGEMENT
# ============================================================================

class BacktestState:
    """Centralized state management for the backtest system"""
    def __init__(self):
        self.uploaded_files = {}  # {filename: dataframe}
        self.processed_data = {}  # {filename: processed_dataframe}
        self.strategy_config = None
        self.all_results = []
        self.results_df = None
        
    def reset(self):
        """Reset all state"""
        self.__init__()
        
    def add_file(self, filename, content):
        """Add uploaded file"""
        self.uploaded_files[filename] = content
        
    def get_file_count(self):
        """Get count of uploaded files"""
        return len(self.uploaded_files)
    
    def get_filenames(self):
        """Get list of uploaded filenames"""
        return list(self.uploaded_files.keys())

# Initialize global state
state = BacktestState()

# ============================================================================
# DATA PREPROCESSING
# ============================================================================

def preprocess_data(df):
    """Enhanced data preprocessing matching your original logic"""
    
    # Display settings
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', 100)
    
    # Ensure required columns exist
    required_cols = ['expiry_date', 'date_only', 'time', 'symbol', 'open', 'high', 'low', 'close', 'index_close']
    missing_cols = [col for col in required_cols if col not in df.columns]
    
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Convert date columns
    df['expiry_date'] = pd.to_datetime(df['expiry_date']).dt.date
    df['date_only'] = pd.to_datetime(df['date_only']).dt.date
    
    # Calculate days to expiry
    df['days_to_expiry'] = (pd.to_datetime(df['expiry_date']) - pd.to_datetime(df['date_only'])).dt.days
    
    # Filter for weekly options (DTE <= 6)
    df = df[df['days_to_expiry'] <= 6].copy()
    
    # Select required columns
    backtest_data = df[['expiry_date', "date_only", 'days_to_expiry', 'time', 'symbol', 
                        'open', 'high', 'low', 'close', 'index_close']].copy()
    
    # Calculate dte_format dynamically based on unique days_to_expiry values
    unique_dte_values = sorted(backtest_data['days_to_expiry'].unique())
    dte_mapping = {dte_value: f'{i} DTE' for i, dte_value in enumerate(unique_dte_values)}
    backtest_data['dte_format'] = backtest_data['days_to_expiry'].map(dte_mapping)
    
    return backtest_data

# ============================================================================
# STRATEGY CONFIGURATION
# ============================================================================

def create_strategy_config(backtest_data):
    """Create strategy configuration based on available DTEs"""
    
    # Get unique DTEs
    unique_dte = sorted(backtest_data['dte_format'].dropna().unique(), reverse=True)
    
    # Create strategy configuration
    strategy_config = pd.DataFrame({
        'DTE': unique_dte,
        'Stoploss %': [60, 60, 60, 60, 60, 50, 50][:len(unique_dte)],
        'Profit Target %': [50, 50, 50, 50, 50, 50, 50][:len(unique_dte)],
        'lot_size': [1, 1, 1, 1, 1, 1, 1][:len(unique_dte)],
        'PE Strike': [4, 4, 4, 4, 4, 4, 4][:len(unique_dte)],
        'CE Strike': [4, 4, 4, 4, 4, 4, 4][:len(unique_dte)],
        'Index Movement +/-': [100, 100, 100, 200, 200, 150, 100][:len(unique_dte)]
    })
    
    # Get index_close at 9:15 for each DTE
    def get_index_close_at_915(dte):
        filtered = backtest_data[
            (backtest_data['dte_format'] == dte) &
            (backtest_data['time'] == '09:15')
        ]
        if not filtered.empty:
            return filtered['index_close'].iloc[0]
        return np.nan
    
    strategy_config['Index Close'] = strategy_config['DTE'].apply(get_index_close_at_915)
    strategy_config['Index Close'] = (strategy_config['Index Close'] / 50).round() * 50
    
    return strategy_config

# ============================================================================
# CORE STRATEGY EXECUTION (FROM YOUR ORIGINAL CODE)
# ============================================================================

def execute_strategy(strategy_row, backtest_data, strategy_config, filename):
    """
    Execute strategy for a given DTE configuration with carry forward and re-entry logic
    (Your original logic - preserved exactly)
    """
    
    initial_dte = strategy_row['DTE']
    
    # Track all trades
    all_trades = []
    
    # Get all unique DTEs in descending order
    all_dtes = sorted(backtest_data['dte_format'].dropna().unique(), reverse=True)
    
    # Start from initial DTE
    current_dte_index = all_dtes.index(initial_dte)
    current_date = None
    current_time = None
    last_exit_index_close = None
    continue_trading = True
    trade_number = 1
    
    # Trading loop
    while continue_trading and current_dte_index < len(all_dtes):
        
        # Get current DTE for entry
        entry_dte = all_dtes[current_dte_index]
        entry_strategy = strategy_config[strategy_config['DTE'] == entry_dte].iloc[0]
        
        # Get entry parameters
        pe_strike_offset = entry_strategy['PE Strike']
        ce_strike_offset = entry_strategy['CE Strike']
        lot_size = entry_strategy['lot_size']
        
        # Determine index close for strike calculation
        if last_exit_index_close is not None:
            index_close_for_entry = last_exit_index_close
        else:
            index_close_for_entry = entry_strategy['Index Close']
        
        # Round to nearest 50
        index_close_for_entry = round(index_close_for_entry / 50) * 50
        
        # Calculate actual strike prices
        pe_strike = index_close_for_entry - (pe_strike_offset * 50)
        ce_strike = index_close_for_entry + (ce_strike_offset * 50)
        
        # Create symbol patterns
        pe_symbol = f"{int(pe_strike)}-PE"
        ce_symbol = f"{int(ce_strike)}-CE"
        
        # Filter backtest data for this DTE
        dte_data = backtest_data[backtest_data['dte_format'] == entry_dte].copy()
        
        # Get entry point
        if current_date is None and current_time is None:
            # First entry at 9:15 AM
            pe_entry = dte_data[
                (dte_data['symbol'].str.contains(pe_symbol, na=False)) &
                (dte_data['time'] == '09:15')
            ]
            ce_entry = dte_data[
                (dte_data['symbol'].str.contains(ce_symbol, na=False)) &
                (dte_data['time'] == '09:15')
            ]
        else:
            # Re-entry logic
            pe_candidates = dte_data[
                (dte_data['symbol'].str.contains(pe_symbol, na=False))
            ].copy()
            
            ce_candidates = dte_data[
                (dte_data['symbol'].str.contains(ce_symbol, na=False))
            ].copy()
            
            if pe_candidates.empty or ce_candidates.empty:
                current_dte_index += 1
                current_date = None
                current_time = None
                last_exit_index_close = None
                continue
            
            # Create datetime for filtering
            pe_candidates['datetime_check'] = pd.to_datetime(
                pe_candidates['date_only'].astype(str) + ' ' + pe_candidates['time'].astype(str)
            )
            ce_candidates['datetime_check'] = pd.to_datetime(
                ce_candidates['date_only'].astype(str) + ' ' + ce_candidates['time'].astype(str)
            )
            
            exit_datetime_str = f"{current_date} {current_time}"
            exit_datetime = pd.to_datetime(exit_datetime_str)
            
            # Filter for rows AFTER exit time
            pe_entry = pe_candidates[pe_candidates['datetime_check'] > exit_datetime].copy()
            ce_entry = ce_candidates[ce_candidates['datetime_check'] > exit_datetime].copy()
            
            if pe_entry.empty or ce_entry.empty:
                current_dte_index += 1
                current_date = None
                current_time = None
                last_exit_index_close = None
                continue
            
            pe_entry = pe_entry.iloc[[0]]
            ce_entry = ce_entry.iloc[[0]]
            
            pe_entry = pe_entry.drop(columns=['datetime_check'])
            ce_entry = ce_entry.drop(columns=['datetime_check'])
        
        if pe_entry.empty or ce_entry.empty:
            break
        
        # Get entry prices and details
        pe_entry_price = pe_entry.iloc[0]['close']
        ce_entry_price = ce_entry.iloc[0]['close']
        total_entry_price = pe_entry_price + ce_entry_price
        entry_date = pe_entry.iloc[0]['date_only']
        entry_time = pe_entry.iloc[0]['time']
        entry_index_close = pe_entry.iloc[0]['index_close']
        
        # Monitor across DTEs until exit
        trade_exited = False
        
        for dte_idx in range(current_dte_index, len(all_dtes)):
            current_monitoring_dte = all_dtes[dte_idx]
            
            # Get strategy parameters for current monitoring DTE
            current_strategy = strategy_config[strategy_config['DTE'] == current_monitoring_dte].iloc[0]
            current_stoploss_pct = current_strategy['Stoploss %']
            current_profit_target_pct = current_strategy['Profit Target %']
            current_index_movement = current_strategy['Index Movement +/-']
            
            # Calculate current targets
            stoploss_price = total_entry_price * (1 + current_stoploss_pct/100)
            profit_target_price = total_entry_price * (1 - current_profit_target_pct/100)
            index_upper_limit = entry_index_close + current_index_movement
            index_lower_limit = entry_index_close - current_index_movement
            
            # Get DTE data
            dte_data = backtest_data[backtest_data['dte_format'] == current_monitoring_dte].copy()
            
            # Filter monitoring data
            pe_monitoring = dte_data[
                (dte_data['symbol'].str.contains(pe_symbol, na=False))
            ].copy()
            
            ce_monitoring = dte_data[
                (dte_data['symbol'].str.contains(ce_symbol, na=False))
            ].copy()
            
            if pe_monitoring.empty or ce_monitoring.empty:
                continue
            
            # Create datetime column for proper time filtering
            pe_monitoring['datetime_check'] = pd.to_datetime(
                pe_monitoring['date_only'].astype(str) + ' ' + pe_monitoring['time'].astype(str)
            )
            ce_monitoring['datetime_check'] = pd.to_datetime(
                ce_monitoring['date_only'].astype(str) + ' ' + ce_monitoring['time'].astype(str)
            )
            
            # Create entry datetime for filtering
            entry_datetime = pd.to_datetime(f"{entry_date} {entry_time}")
            
            # Filter for data >= entry time
            pe_monitoring = pe_monitoring[pe_monitoring['datetime_check'] >= entry_datetime].copy()
            ce_monitoring = ce_monitoring[ce_monitoring['datetime_check'] >= entry_datetime].copy()
            
            if pe_monitoring.empty or ce_monitoring.empty:
                continue
            
            # Drop datetime_check before merge
            pe_monitoring = pe_monitoring.drop(columns=['datetime_check'])
            ce_monitoring = ce_monitoring.drop(columns=['datetime_check'])
            
            # Merge PE and CE data
            monitoring_data = pd.merge(
                pe_monitoring[['date_only', 'time', 'close', 'index_close']],
                ce_monitoring[['date_only', 'time', 'close']],
                on=['date_only', 'time'],
                suffixes=('_pe', '_ce')
            )
            
            if monitoring_data.empty:
                continue
            
            # Calculate combined position value
            monitoring_data['total_close'] = monitoring_data['close_pe'] + monitoring_data['close_ce']
            
            # Check for exit conditions
            for idx, row in monitoring_data.iterrows():
                # Skip entry time point
                if row['date_only'] == entry_date and row['time'] == entry_time:
                    continue
                
                # Check if 0 DTE and time is 15:20 (FINAL EXIT)
                if current_monitoring_dte == '0 DTE' and row['time'] == '13:20':
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': '0 DTE Expiry (3:20 PM)',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    continue_trading = False
                    trade_exited = True
                    break
                
                # Check stoploss breach
                if row['total_close'] >= stoploss_price:
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': 'Stoploss Hit',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    
                    current_date = exit_date
                    current_time = exit_time
                    last_exit_index_close = exit_index_close
                    current_dte_index = all_dtes.index(current_monitoring_dte)
                    trade_exited = True
                    trade_number += 1
                    break
                
                # Check profit target breach
                if row['total_close'] <= profit_target_price:
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': 'Profit Target Hit',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    
                    current_date = exit_date
                    current_time = exit_time
                    last_exit_index_close = exit_index_close
                    current_dte_index = all_dtes.index(current_monitoring_dte)
                    trade_exited = True
                    trade_number += 1
                    break
                
                # Check index movement breach
                if row['index_close'] >= index_upper_limit or row['index_close'] <= index_lower_limit:
                    exit_time = row['time']
                    exit_date = row['date_only']
                    exit_price = row['total_close']
                    exit_index_close = row['index_close']
                    pnl = (total_entry_price - exit_price) * lot_size * 50
                    
                    trade_result = {
                        'Filename': filename,
                        'Trade_Number': trade_number,
                        'Entry_DTE': entry_dte,
                        'Exit_DTE': current_monitoring_dte,
                        'Entry_Date': entry_date,
                        'Entry_Time': entry_time,
                        'Exit_Date': exit_date,
                        'Exit_Time': exit_time,
                        'PE_Strike': pe_strike,
                        'CE_Strike': ce_strike,
                        'PE_Entry_Price': pe_entry_price,
                        'CE_Entry_Price': ce_entry_price,
                        'Total_Entry_Price': total_entry_price,
                        'Exit_Price': exit_price,
                        'Exit_Reason': 'Index Movement Breach',
                        'PnL': pnl,
                        'Entry_Index_Close': entry_index_close,
                        'Exit_Index_Close': exit_index_close
                    }
                    
                    all_trades.append(trade_result)
                    
                    current_date = exit_date
                    current_time = exit_time
                    last_exit_index_close = exit_index_close
                    current_dte_index = all_dtes.index(current_monitoring_dte)
                    trade_exited = True
                    trade_number += 1
                    break
            
            if trade_exited:
                break
        
        if not trade_exited:
            break
    
    return all_trades

# ============================================================================
# UI COMPONENTS
# ============================================================================

def create_file_upload_ui():
    """Create file upload interface"""
    
    upload_widget = widgets.FileUpload(
        accept='.csv,.xlsx,.xls',
        multiple=True,
        description='Upload Files',
        button_style='primary',
        layout=widgets.Layout(width='300px')
    )
    
    file_list_output = widgets.Output()
    upload_status = widgets.HTML()
    
    def on_upload_change(change):
        """Handle file upload"""
        with file_list_output:
            clear_output()
            
            uploaded = change['new']
            if not uploaded:
                return
            
            # Handle tuple format from FileUpload widget
            for file_info in uploaded:
                try:
                    filename = file_info['name']
                    content = file_info['content']
                    
                    # Read file based on extension
                    if filename.endswith('.csv'):
                        df = pd.read_csv(io.BytesIO(content))
                    elif filename.endswith(('.xlsx', '.xls')):
                        df = pd.read_excel(io.BytesIO(content))
                    else:
                        print(f"❌ Unsupported file format: {filename}")
                        continue
                    
                    # Store in state
                    state.add_file(filename, df)
                    print(f"✅ Uploaded: {filename} ({len(df)} rows)")
                    
                except Exception as e:
                    print(f"❌ Error uploading {filename}: {str(e)}")
            
            # Update status
            upload_status.value = f'<p style="color: green;">📁 {state.get_file_count()} files uploaded successfully</p>'
    
    upload_widget.observe(on_upload_change, names='value')
    
    return widgets.VBox([
        widgets.HTML('<h3 style="color: #2c3e50;">📁 Step 1: Upload Weekly Expiry Files</h3>'),
        widgets.HTML('<p style="color: #7f8c8d;">Upload multiple CSV or Excel files containing weekly options data</p>'),
        upload_widget,
        upload_status,
        file_list_output
    ])

def create_strategy_config_ui():
    """Create strategy configuration interface"""
    
    process_btn = widgets.Button(
        description='🔧 Process Files & Configure Strategy',
        button_style='warning',
        layout=widgets.Layout(width='300px')
    )
    
    config_output = widgets.Output()
    strategy_widgets = {}
    
    def on_process_click(b):
        """Process uploaded files and create strategy config"""
        with config_output:
            clear_output()
            
            if state.get_file_count() == 0:
                print("❌ Please upload files first!")
                return
            
            print("🔧 Processing uploaded files...")
            
            # Process first file to get DTE structure
            first_filename = state.get_filenames()[0]
            first_df = state.uploaded_files[first_filename]
            
            try:
                processed = preprocess_data(first_df)
                state.processed_data[first_filename] = processed
                
                # Get unique DTEs
                unique_dte = sorted(processed['dte_format'].dropna().unique(), reverse=True)
                
                print(f"✅ Detected {len(unique_dte)} DTEs: {unique_dte}")
                print("\n⚙️ Configure Strategy Parameters:\n")
                
                # Create configuration widgets
                display(HTML('<table style="width:100%; border-collapse: collapse;"><tr style="background: #3498db; color: white;"><th>DTE</th><th>Stoploss %</th><th>Profit Target %</th><th>Lot Size</th><th>PE Offset</th><th>CE Offset</th><th>Index Move</th></tr></table>'))
                
                for i, dte in enumerate(unique_dte):
                    row_widgets = {
                        'dte_label': widgets.Label(value=dte, layout=widgets.Layout(width='80px')),
                        'sl_pct': widgets.FloatText(value=30 + (i * 5), layout=widgets.Layout(width='100px')),
                        'profit_pct': widgets.FloatText(value=50 - (i * 5), layout=widgets.Layout(width='100px')),
                        'lot_size': widgets.IntText(value=1, layout=widgets.Layout(width='80px')),
                        'pe_strike': widgets.IntText(value=0 if i < 3 else 3, layout=widgets.Layout(width='80px')),
                        'ce_strike': widgets.IntText(value=0 if i < 3 else 3, layout=widgets.Layout(width='80px')),
                        'index_move': widgets.IntText(value=100, layout=widgets.Layout(width='100px'))
                    }
                    
                    strategy_widgets[dte] = row_widgets
                    
                    display(widgets.HBox([
                        row_widgets['dte_label'],
                        row_widgets['sl_pct'],
                        row_widgets['profit_pct'],
                        row_widgets['lot_size'],
                        row_widgets['pe_strike'],
                        row_widgets['ce_strike'],
                        row_widgets['index_move']
                    ]))
                
                # Store widgets globally
                globals()['strategy_widgets'] = strategy_widgets
                
                print("\n✅ Strategy configuration ready!")
                print("🎯 Proceed to execute backtest below")
                
            except Exception as e:
                print(f"❌ Error processing files: {str(e)}")
                import traceback
                traceback.print_exc()
    
    process_btn.on_click(on_process_click)
    
    return widgets.VBox([
        widgets.HTML('<h3 style="color: #2c3e50;">⚙️ Step 2: Process Files & Configure Strategy</h3>'),
        process_btn,
        config_output
    ])

def create_backtest_execution_ui():
    """Create backtest execution interface"""
    
    run_btn = widgets.Button(
        description='🚀 Run Backtest',
        button_style='success',
        layout=widgets.Layout(width='200px', height='50px'),
        icon='play'
    )
    
    progress_output = widgets.Output()
    results_output = widgets.Output()
    
    def on_run_click(b):
        """Execute backtest on all files"""
        with progress_output:
            clear_output()
            
            if state.get_file_count() == 0:
                print("❌ Please upload files first!")
                return
            
            if 'strategy_widgets' not in globals():
                print("❌ Please configure strategy first!")
                return
            
            print("="*80)
            print("🚀 STARTING MULTI-FILE BACKTEST")
            print("="*80)
            print(f"📊 Total Files: {state.get_file_count()}\n")
            
            all_results = []
            
            # Process each file
            for file_idx, filename in enumerate(state.get_filenames(), 1):
                print(f"\n{'='*80}")
                print(f"📂 Processing File {file_idx}/{state.get_file_count()}: {filename}")
                print(f"{'='*80}")
                
                try:
                    # Get or process data
                    if filename in state.processed_data:
                        backtest_data = state.processed_data[filename]
                    else:
                        df = state.uploaded_files[filename]
                        backtest_data = preprocess_data(df)
                        state.processed_data[filename] = backtest_data
                    
                    print(f"✅ Data loaded: {len(backtest_data)} rows")
                    
                    # Create strategy config from widgets
                    strategy_config_list = []
                    for dte, widgets_dict in strategy_widgets.items():
                        strategy_config_list.append({
                            'DTE': dte,
                            'Stoploss %': widgets_dict['sl_pct'].value,
                            'Profit Target %': widgets_dict['profit_pct'].value,
                            'lot_size': widgets_dict['lot_size'].value,
                            'PE Strike': widgets_dict['pe_strike'].value,
                            'CE Strike': widgets_dict['ce_strike'].value,
                            'Index Movement +/-': widgets_dict['index_move'].value
                        })
                    
                    strategy_config = pd.DataFrame(strategy_config_list)
                    
                    # Get index close at 9:15 for each DTE
                    def get_index_close_at_915(dte):
                        filtered = backtest_data[
                            (backtest_data['dte_format'] == dte) &
                            (backtest_data['time'] == '09:15')
                        ]
                        if not filtered.empty:
                            return filtered['index_close'].iloc[0]
                        return np.nan
                    
                    strategy_config['Index Close'] = strategy_config['DTE'].apply(get_index_close_at_915)
                    strategy_config['Index Close'] = (strategy_config['Index Close'] / 50).round() * 50
                    
                    print("\n📋 Strategy Config:")
                    print(strategy_config.to_string(index=False))
                    
                    # Execute strategy
                    max_dte_strategy = strategy_config.iloc[0]
                    results = execute_strategy(max_dte_strategy, backtest_data, strategy_config, filename)
                    
                    print(f"\n✅ Completed: {len(results)} trades executed")
                    
                    all_results.extend(results)
                    
                except Exception as e:
                    print(f"❌ Error processing {filename}: {str(e)}")
                    import traceback
                    traceback.print_exc()
                    continue
            
            # Display consolidated results
            with results_output:
                clear_output()
                
                if all_results:
                    results_df = pd.DataFrame(all_results)
                    state.results_df = results_df
                    
                    print("\n" + "="*80)
                    print("📊 BACKTEST COMPLETED - CONSOLIDATED RESULTS")
                    print("="*80)
                    
                    # Overall statistics
                    total_pnl = results_df['PnL'].sum()
                    winning_trades = len(results_df[results_df['PnL'] > 0])
                    losing_trades = len(results_df[results_df['PnL'] < 0])
                    win_rate = (winning_trades / len(results_df) * 100) if len(results_df) > 0 else 0
                    
                    print(f"\n🎯 OVERALL STATISTICS:")
                    print(f"{'─'*80}")
                    print(f"Total Files Processed:    {results_df['Filename'].nunique()}")
                    print(f"Total Trades:             {len(results_df)}")
                    print(f"Winning Trades:           {winning_trades}")
                    print(f"Losing Trades:            {losing_trades}")
                    print(f"Win Rate:                 {win_rate:.2f}%")
                    print(f"Total P&L:                ₹{total_pnl:,.2f}")
                    print(f"Average P&L per Trade:    ₹{results_df['PnL'].mean():,.2f}")
                    print(f"Best Trade:               ₹{results_df['PnL'].max():,.2f}")
                    print(f"Worst Trade:              ₹{results_df['PnL'].min():,.2f}")
                    
                    # File-wise summary
                    print(f"\n📂 FILE-WISE SUMMARY:")
                    print(f"{'─'*80}")
                    file_summary = results_df.groupby('Filename').agg({
                        'PnL': ['sum', 'mean', 'count'],
                        'Trade_Number': 'max'
                    }).round(2)
                    file_summary.columns = ['Total PnL', 'Avg PnL', 'Trades', 'Max Trade #']
                    print(file_summary.to_string())
                    
                    # Exit reason analysis
                    print(f"\n🎯 EXIT REASON ANALYSIS:")
                    print(f"{'─'*80}")
                    exit_analysis = results_df.groupby('Exit_Reason').agg({
                        'PnL': ['sum', 'mean', 'count']
                    }).round(2)
                    exit_analysis.columns = ['Total PnL', 'Avg PnL', 'Count']
                    print(exit_analysis.to_string())
                    
                    # Export to Excel with comprehensive analysis
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    excel_filename = f'Backtest_Results_{timestamp}.xlsx'
                    
                    print(f"\n📊 Generating comprehensive Excel report...")
                    
                    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                        
                        # Sheet 1: All Trades - Complete trade log with week & file source
                        all_trades_export = results_df.copy()
                        all_trades_export['Week_Number'] = pd.to_datetime(all_trades_export['Entry_Date']).dt.isocalendar().week
                        all_trades_export['Month'] = pd.to_datetime(all_trades_export['Entry_Date']).dt.to_period('M').astype(str)
                        all_trades_export.to_excel(writer, sheet_name='All_Trades', index=False)
                        print("   ✓ Sheet 1: All_Trades")
                        
                        # Sheet 2: Strategy Summary - Consolidated strategy performance
                        strategy_summary = pd.DataFrame({
                            'Metric': [
                                'Total Files Processed',
                                'Total Trades',
                                'Winning Trades',
                                'Losing Trades',
                                'Win Rate (%)',
                                'Total P&L (₹)',
                                'Average P&L per Trade (₹)',
                                'Average Win (₹)',
                                'Average Loss (₹)',
                                'Best Trade (₹)',
                                'Worst Trade (₹)',
                                'Profit Factor',
                                'Total Days Traded'
                            ],
                            'Value': [
                                results_df['Filename'].nunique(),
                                len(results_df),
                                winning_trades,
                                losing_trades,
                                round(win_rate, 2),
                                round(total_pnl, 2),
                                round(results_df['PnL'].mean(), 2),
                                round(results_df[results_df['PnL'] > 0]['PnL'].mean(), 2) if winning_trades > 0 else 0,
                                round(results_df[results_df['PnL'] < 0]['PnL'].mean(), 2) if losing_trades > 0 else 0,
                                round(results_df['PnL'].max(), 2),
                                round(results_df['PnL'].min(), 2),
                                round((results_df[results_df['PnL'] > 0]['PnL'].sum() / abs(results_df[results_df['PnL'] < 0]['PnL'].sum())), 2) if losing_trades > 0 else float('inf'),
                                results_df['Entry_Date'].nunique()
                            ]
                        })
                        strategy_summary.to_excel(writer, sheet_name='Strategy_Summary', index=False)
                        print("   ✓ Sheet 2: Strategy_Summary")
                        
                        # Sheet 3: Overall Metrics - Multi-week KPIs
                        overall_metrics = results_df.agg({
                            'PnL': ['sum', 'mean', 'std', 'min', 'max', 'count']
                        }).round(2)
                        overall_metrics_df = pd.DataFrame({
                            'Metric': ['Total P&L', 'Average P&L', 'Std Dev P&L', 'Min P&L', 'Max P&L', 'Trade Count'],
                            'Value': overall_metrics.values.flatten()
                        })
                        
                        # Add additional KPIs
                        sharpe = results_df['PnL'].mean() / results_df['PnL'].std() if results_df['PnL'].std() > 0 else 0
                        overall_metrics_df = pd.concat([overall_metrics_df, pd.DataFrame({
                            'Metric': ['Sharpe Ratio', 'Total Files', 'Avg Trades per File'],
                            'Value': [
                                round(sharpe, 2),
                                results_df['Filename'].nunique(),
                                round(len(results_df) / results_df['Filename'].nunique(), 2)
                            ]
                        })], ignore_index=True)
                        overall_metrics_df.to_excel(writer, sheet_name='Overall_Metrics', index=False)
                        print("   ✓ Sheet 3: Overall_Metrics")
                        
                        # Sheet 4: DTE Analysis - Performance by DTE with standard deviation
                        dte_analysis = results_df.groupby('Entry_DTE').agg({
                            'PnL': ['sum', 'mean', 'std', 'count', 'min', 'max']
                        }).round(2)
                        dte_analysis.columns = ['Total_PnL', 'Avg_PnL', 'Std_Dev', 'Trade_Count', 'Min_PnL', 'Max_PnL']
                        dte_analysis['Win_Rate_%'] = results_df.groupby('Entry_DTE').apply(
                            lambda x: round((len(x[x['PnL'] > 0]) / len(x) * 100), 2)
                        )
                        dte_analysis.to_excel(writer, sheet_name='DTE_Analysis')
                        print("   ✓ Sheet 4: DTE_Analysis")
                        
                        # Sheet 5: Exit Analysis - Exit reason breakdown
                        exit_analysis = results_df.groupby('Exit_Reason').agg({
                            'PnL': ['sum', 'mean', 'std', 'count', 'min', 'max']
                        }).round(2)
                        exit_analysis.columns = ['Total_PnL', 'Avg_PnL', 'Std_Dev', 'Count', 'Min_PnL', 'Max_PnL']
                        exit_analysis['Win_Rate_%'] = results_df.groupby('Exit_Reason').apply(
                            lambda x: round((len(x[x['PnL'] > 0]) / len(x) * 100), 2)
                        )
                        exit_analysis.to_excel(writer, sheet_name='Exit_Analysis')
                        print("   ✓ Sheet 5: Exit_Analysis")
                        
                        # Sheet 6: Weekly Performance - Week-by-week results
                        results_with_week = results_df.copy()
                        results_with_week['Week_Number'] = pd.to_datetime(results_with_week['Entry_Date']).dt.isocalendar().week
                        results_with_week['Year'] = pd.to_datetime(results_with_week['Entry_Date']).dt.year
                        results_with_week['Week_ID'] = results_with_week['Year'].astype(str) + '-W' + results_with_week['Week_Number'].astype(str).str.zfill(2)
                        
                        weekly_performance = results_with_week.groupby('Week_ID').agg({
                            'PnL': ['sum', 'mean', 'std', 'count', 'min', 'max'],
                            'Filename': 'first',
                            'Trade_Number': 'max'
                        }).round(2)
                        weekly_performance.columns = ['Total_PnL', 'Avg_PnL', 'Std_Dev', 'Trades', 'Min_PnL', 'Max_PnL', 'Filename', 'Max_Trade_Num']
                        weekly_performance['Win_Rate_%'] = results_with_week.groupby('Week_ID').apply(
                            lambda x: round((len(x[x['PnL'] > 0]) / len(x) * 100), 2)
                        )
                        weekly_performance.to_excel(writer, sheet_name='Weekly_Performance')
                        print("   ✓ Sheet 6: Weekly_Performance")
                        
                        # Sheet 7: Monthly Performance - Monthly aggregation
                        results_with_month = results_df.copy()
                        results_with_month['Month'] = pd.to_datetime(results_with_month['Entry_Date']).dt.to_period('M')
                        
                        monthly_performance = results_with_month.groupby('Month').agg({
                            'PnL': ['sum', 'mean', 'std', 'count', 'min', 'max']
                        }).round(2)
                        monthly_performance.columns = ['Total_PnL', 'Avg_PnL', 'Std_Dev', 'Trades', 'Min_PnL', 'Max_PnL']
                        monthly_performance['Win_Rate_%'] = results_with_month.groupby('Month').apply(
                            lambda x: round((len(x[x['PnL'] > 0]) / len(x) * 100), 2)
                        )
                        monthly_performance.index = monthly_performance.index.astype(str)
                        monthly_performance.to_excel(writer, sheet_name='Monthly_Performance')
                        print("   ✓ Sheet 7: Monthly_Performance")
                        
                        # Sheet 8: Cumulative P&L - With week tracking
                        cumulative_data = results_df.sort_values(['Filename', 'Entry_Date', 'Entry_Time']).copy()
                        cumulative_data['Cumulative_PnL'] = cumulative_data['PnL'].cumsum()
                        cumulative_data['Week_Number'] = pd.to_datetime(cumulative_data['Entry_Date']).dt.isocalendar().week
                        cumulative_data['Year'] = pd.to_datetime(cumulative_data['Entry_Date']).dt.year
                        cumulative_data['Week_ID'] = cumulative_data['Year'].astype(str) + '-W' + cumulative_data['Week_Number'].astype(str).str.zfill(2)
                        
                        cumulative_export = cumulative_data[[
                            'Filename', 'Week_ID', 'Entry_Date', 'Entry_Time', 
                            'Trade_Number', 'PnL', 'Cumulative_PnL', 'Exit_Reason'
                        ]].copy()
                        cumulative_export.to_excel(writer, sheet_name='Cumulative_PnL', index=False)
                        print("   ✓ Sheet 8: Cumulative_PnL")
                        
                        # Sheet 9: Drawdown Analysis - With week context
                        cumulative_sorted = results_df.sort_values(['Filename', 'Entry_Date', 'Entry_Time']).copy()
                        cumulative_sorted['Cumulative_PnL'] = cumulative_sorted['PnL'].cumsum()
                        cumulative_sorted['Running_Max'] = cumulative_sorted['Cumulative_PnL'].cummax()
                        cumulative_sorted['Drawdown'] = cumulative_sorted['Cumulative_PnL'] - cumulative_sorted['Running_Max']
                        cumulative_sorted['Drawdown_%'] = (cumulative_sorted['Drawdown'] / cumulative_sorted['Running_Max'].abs()) * 100
                        cumulative_sorted['Drawdown_%'] = cumulative_sorted['Drawdown_%'].fillna(0)
                        
                        cumulative_sorted['Week_Number'] = pd.to_datetime(cumulative_sorted['Entry_Date']).dt.isocalendar().week
                        cumulative_sorted['Year'] = pd.to_datetime(cumulative_sorted['Entry_Date']).dt.year
                        cumulative_sorted['Week_ID'] = cumulative_sorted['Year'].astype(str) + '-W' + cumulative_sorted['Week_Number'].astype(str).str.zfill(2)
                        
                        drawdown_export = cumulative_sorted[[
                            'Filename', 'Week_ID', 'Entry_Date', 'Trade_Number',
                            'PnL', 'Cumulative_PnL', 'Running_Max', 'Drawdown', 'Drawdown_%'
                        ]].copy()
                        drawdown_export.to_excel(writer, sheet_name='Drawdown_Analysis', index=False)
                        print("   ✓ Sheet 9: Drawdown_Analysis")
                        
                        # Add summary statistics for drawdown
                        max_drawdown = cumulative_sorted['Drawdown'].min()
                        max_drawdown_pct = cumulative_sorted['Drawdown_%'].min()
                        max_dd_date = cumulative_sorted.loc[cumulative_sorted['Drawdown'].idxmin(), 'Entry_Date']
                        max_dd_week = cumulative_sorted.loc[cumulative_sorted['Drawdown'].idxmin(), 'Week_ID']
                        
                        drawdown_summary = pd.DataFrame({
                            'Metric': [
                                'Max Drawdown (₹)',
                                'Max Drawdown (%)',
                                'Max DD Date',
                                'Max DD Week',
                                'Recovery Trades',
                                'Avg Drawdown (₹)',
                                'Drawdown Days'
                            ],
                            'Value': [
                                round(max_drawdown, 2),
                                round(max_drawdown_pct, 2),
                                str(max_dd_date),
                                max_dd_week,
                                len(cumulative_sorted[cumulative_sorted['Drawdown'] < 0]),
                                round(cumulative_sorted[cumulative_sorted['Drawdown'] < 0]['Drawdown'].mean(), 2),
                                cumulative_sorted[cumulative_sorted['Drawdown'] < 0]['Entry_Date'].nunique()
                            ]
                        })
                        
                        # Append drawdown summary to the same sheet
                        startrow = len(drawdown_export) + 3
                        drawdown_summary.to_excel(writer, sheet_name='Drawdown_Analysis', 
                                                  startrow=startrow, index=False)
                    
                    print(f"\n✅ Comprehensive Excel report generated: {excel_filename}")
                    print(f"📊 Total sheets: 9")
                    print("\n💾 Results stored in 'state.results_df' for further analysis")
                    
                else:
                    print("\n❌ No trades executed across all files")
    
    run_btn.on_click(on_run_click)
    
    return widgets.VBox([
        widgets.HTML('<h3 style="color: #2c3e50;">🚀 Step 3: Execute Backtest</h3>'),
        run_btn,
        progress_output,
        results_output
    ])

# ============================================================================
# MAIN UI ASSEMBLY
# ============================================================================

def create_main_ui():
    """Assemble main user interface"""
    
    header = widgets.HTML('''
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                    padding: 30px; 
                    border-radius: 10px; 
                    text-align: center;
                    margin-bottom: 20px;">
            <h1 style="color: white; margin: 0; font-size: 36px;">📈 Options Backtest System</h1>
            <p style="color: #e0e0e0; margin: 10px 0 0 0; font-size: 18px;">
                Multi-File Strategy Backtesting with Re-Entry Logic
            </p>
        </div>
    ''')
    
    divider = widgets.HTML('<hr style="border: 2px solid #3498db; margin: 30px 0;">')
    
    file_upload_ui = create_file_upload_ui()
    strategy_config_ui = create_strategy_config_ui()
    backtest_execution_ui = create_backtest_execution_ui()
    
    main_ui = widgets.VBox([
        header,
        file_upload_ui,
        divider,
        strategy_config_ui,
        divider,
        backtest_execution_ui
    ])
    
    return main_ui

# ============================================================================
# LAUNCH APPLICATION
# ============================================================================

def launch_backtest_system():
    """Launch the backtest system"""
    
    print("✅ Options Backtest System Initialized")
    print("\n📋 Features:")
    print("   • Multiple file upload (CSV/Excel)")
    print("   • Dynamic DTE-based strategy configuration")
    print("   • Re-entry logic after SL/Profit exits")
    print("   • Carry forward across DTEs until 0 DTE expiry")
    print("   • File-wise and consolidated performance tracking")
    print("   • Excel export with multiple analysis sheets")
    print("\n🎯 Instructions:")
    print("   1. Upload your weekly expiry CSV/Excel files")
    print("   2. Process files and configure strategy parameters")
    print("   3. Run backtest to get consolidated results")
    print("\n" + "="*80 + "\n")
    
    main_ui = create_main_ui()
    display(main_ui)

# Auto-launch when imported
if __name__ == '__main__':
    launch_backtest_system()


✅ Options Backtest System Initialized

📋 Features:
   • Multiple file upload (CSV/Excel)
   • Dynamic DTE-based strategy configuration
   • Re-entry logic after SL/Profit exits
   • Carry forward across DTEs until 0 DTE expiry
   • File-wise and consolidated performance tracking
   • Excel export with multiple analysis sheets

🎯 Instructions:
   1. Upload your weekly expiry CSV/Excel files
   2. Process files and configure strategy parameters
   3. Run backtest to get consolidated results


